In [ ]:
# Texto de exemplo
with open('macmorpho-dev.txt', 'r', encoding='utf-8') as arquivo:
    text = arquivo.read()

Tentei treinar o BERT, mas sempre dava erro de memória. Pelo visto, as operações com o corpus estavam muito grandes pro meu computador aguentar (ele tem apenas 500GB de memória ), dessa forma, tive que optar por não treinar o modelo.

In [ ]:
def extract_tag_after_underscore(frase):
    palavras = [palavra.split('_')[1] for palavra in frase.split() if '_' in palavra]
    return palavras

existing_tags = extract_tag_after_underscore(text)

print("tags:", existing_tags)

In [ ]:
def extract_tag_before_underscore(frase):
    palavras = [palavra.split('_')[0] for palavra in frase.split() if '_' in palavra]
    return palavras

sentences_without_separation = extract_tag_before_underscore(text)
sentences_temp = ['[SEP]' if (elemento == ',' or elemento == ';') else elemento for elemento in sentences_without_separation]
sentences = ['[CLS]' if (elemento == '.') else elemento for elemento in sentences_temp]
# Adicionando o começo da primeira sentença
sentences.insert(0, '[CLS]')


print("tags:", sentences)

In [ ]:
# to remove duplicate tags

import numpy as np 
import pandas as pd 

df = pd.DataFrame(existing_tags)
df.duplicated().sum()
df.drop_duplicates(inplace=True)
df.columns = ['tag']
df.head()

In [ ]:

df.loc[df.shape[0] - 1 , 'tag'] = '[CLS]'
df.loc[df.shape[0] - 1 , 'tag'] = '[SEP]'
tag_values = df['tag'].values
print(tag_values)
len(tag_values)

In [ ]:
tag_map = {rotulo: indice for indice, rotulo in enumerate(tag_values)}
print(tag_map)

Minha intenção era treinar o BERT, mas sempre dava erro de memória. Pelo visto, as operações com o corpus estavam muito grandes pro meu computador aguentar (ele tem apenas 500GB de memória ), dessa forma, tive que optar por utilizar um modelo pré-treinado.

O primeiro modelo testado foi o 'bert-base-multilingual-cased', o qual tem o modelo pré-treinado com diversas línguas. A acurácia foi de apenas 2.93%, enquanto a precisão, 5.99%.

Mudando pra as opções em português, os resultados foram melhores. Segue as opções testadas e resultados:
- 'neuralmind/bert-base-portuguese-cased': Acurácia: 7.03%, Precisão: 20.16%
- 'neuralmind/bert-large-portuguese-cased': Acurácia: 5.27%, Precisão: 7.29%
- 'pierreguillou/bert-base-cased-squad-v1.1-portuguese': Acurácia: 0.59%, Precisão: 0.00%

In [ ]:
from transformers import BertTokenizer, BertForTokenClassification
import torch

# Seu corpus
corpus = text

# Lista de tags
tag2idx = tag_map

# Tokenizar o corpus
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
# tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sentences, add_special_tokens=True)))
tokens = tokenizer.encode_plus(sentences, max_length=10, padding='max_length', truncation=False, add_special_tokens=True)
tokens = tokens[:512]

# Generate attention mask
attention_mask = tokens['attention_mask']

# Converter as tags em índices
all_tags = extract_tag_after_underscore(corpus)
list_numbers = [tag2idx[elemento] for elemento in all_tags]
tag_ids = list_numbers
tag_ids = tag_ids[:512]

# Converter tokens em IDs
input_ids = tokenizer.encode(sentences, add_special_tokens=False)
input_ids_orig = input_ids[:512]


In [ ]:
print(tag_ids)
print(input_ids)
print(len(attention_mask))
print(len(tag_ids))
print(len(input_ids))

In [ ]:
print(tokens['input_ids'])

In [ ]:
print(tokens)

Com resultados baixos, decidiu utilizar métodos de otimização pra ajustar hiperparâmetros. Segue os resultados testados:

- Taxa de aprendizado: o melhor resultado foi colocar lr=1e-3, no qual obteve Acurácia: 38.67%, Precisão: 31.41%. Esse resultado teve um salto em relação ao anterior, com Acurácia de 7.03% e Precisão: 20.16%.

- Epochs: Essa variável foi o que mais influenciou no resultado. Anteriormente, só 3 épocas eram rodadas, mas ao setar para 50, a acurácia foi de 85.16% e a precisão: 87.09%, melhorando bastante o resultado. Entretanto, percebeu-se que o erro médio caía muito até mais ou menos a época 33, a partir dali, começou a crescer. Há grande possibilidade de overfitting.

In [ ]:
# Model adjustments
from transformers import AdamW, BertConfig
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn


config = BertConfig.from_pretrained('neuralmind/bert-base-portuguese-cased')
config.hidden_dropout_prob = 0.4  # Exemplo de dropout na camada de embedding
config.attention_probs_dropout_prob = 0.4  # Exemplo de dropout nas camadas de atenção

# Carregar o modelo BERT para token classification
model = BertForTokenClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=len(tag2idx))
model.classifier.dropout = nn.Dropout(0.4)

optimizer = AdamW(model.parameters(), lr=1e-3, weight_decay=0.1)

# Dados para DataLoader
labels = all_tags[:512]
labels = [tag2idx[label] for label in labels]

input_ids = torch.tensor(input_ids_orig)
attention_mask = torch.tensor(attention_mask)
labels = torch.tensor(labels)

input_ids = input_ids.unsqueeze(0)
attention_mask = attention_mask.unsqueeze(0)
labels = labels.unsqueeze(0)

dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Treinamento
epochs = 50  # Ajuste conforme necessário
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in dataloader:
        batch = tuple(t.to(device) for t in batch)
        inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
        outputs = model(**inputs)
        loss = outputs.loss
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}, Average Loss: {avg_loss}")

In [ ]:
len(input_ids_orig)
len(tag_ids)

In [ ]:
# Realizar a inferência
with torch.no_grad():
    outputs = model(torch.tensor([input_ids_orig]), labels=torch.tensor([tag_ids]))

# Obter as previsões
predictions = torch.argmax(outputs.logits, dim=2).tolist()[0]

# Mapear os índices de volta para tags
idx2tag = {idx: tag for tag, idx in tag2idx.items()}
predicted_tags = [idx2tag[idx] for idx in predictions]

In [ ]:
# Exibir o resultado
tokens_from_ids = tokenizer.convert_ids_to_tokens(input_ids[0].tolist())

for token, tag in zip(tokens_from_ids, predicted_tags):
    print(f"Token: {token}\t\tPOS: {tag}")

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calcular acurácia
accuracy = accuracy_score(all_tags[:512], predicted_tags)
precision = precision_score(all_tags[:512], predicted_tags, average='weighted')
recall = recall_score(all_tags[:512], predicted_tags, average='weighted')
f1 = f1_score(all_tags[:512], predicted_tags, average='weighted')

print(f'Acurácia: {accuracy * 100:.2f}%')
print(f'Precisão: {precision * 100:.2f}%')
print(f'Recall: {recall * 100:.2f}%')
print(f'F1-score: {f1 * 100:.2f}%')